In [1]:
#Imports

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
import math
import random
import time
import codecs

from itertools import *

%matplotlib inline

In [2]:
data_file = '../../data/lorca.txt'

data = codecs.open(data_file, 'rU', 'utf-8').read() # AH: Included unicode support through module 'codecs'
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'Data has %d characters, %d unique.' % (data_size, vocab_size)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
seq_length = 10 # number of steps to unroll the RNN for


Data has 82702 characters, 72 unique.


In [3]:
n, p = 0, 0
n_max = 100000
inputs = []
targets = []

while n <= n_max:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        p = 0 # go from start of data
    input = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    target = [char_to_ix[ch] for ch in data[p+seq_length:p+seq_length+1]]
    p += 1
    n += 1
    inputs.append(input)
    targets.append(target)

In [4]:
# Targets to one hot encoding
targets_array = []
for target in targets:
    _target = target[0]
    target_array = np.zeros(72) # AH: OJO, esto debería ser vocab_size
    target_array[_target] = 1
    targets_array.append(target_array)
targets = np.array(targets_array)

In [5]:
# Inputs to one hot encoding
inputs_arrays = []
for input in inputs:
    input_array = []
    for input_value in input:
        input_ohv = np.zeros(72) # AH: OJO, esto debería ser vocab_size
        input_ohv[input_value] = 1
        input_array.append(input_ohv)
    input_array = np.array(input_array)
    inputs_arrays.append(input_array)
inputs = np.array(inputs_arrays)

In [6]:
input_data = np.array(inputs)
output_data = np.array(targets)

print input_data.shape
print output_data.shape

(100001, 10, 72)
(100001, 72)


In [7]:
# Reshape the data from rows to columns
# AH: No estoy seguro de lo que hace esto; parecen quedar con igual shape.
input_data = input_data.reshape(input_data.shape[0], seq_length, 72)
output_data = output_data.reshape(output_data.shape[0], 72)

print input_data.shape
print output_data.shape

(100001, 10, 72)
(100001, 72)


In [8]:
# Data Split
# AH: con LIMIT a 800 se pierde mucho dato de entrenamiento en sets grandes
# AH: se podría probar con el 80% de input_data.shape[0].
LIMIT = 800
X_test = input_data[LIMIT:]
Y_test = output_data[LIMIT:]

X_train = input_data[:LIMIT]
Y_train = output_data[:LIMIT]

In [9]:
# Network variables

data = tf.placeholder(tf.float32, [None, seq_length, 72])
target = tf.placeholder(tf.float32, [None, 72])

num_hidden = 2
num_neurons = 50

cell = tf.nn.rnn_cell.LSTMCell(num_neurons, state_is_tuple=False)
stacked_lstm = tf.nn.rnn_cell.MultiRNNCell([cell] * num_hidden, state_is_tuple=False)
val, state = tf.nn.dynamic_rnn(stacked_lstm, data, dtype=tf.float32)

# TODO: Understand what happens in the next couple of lines!!!
val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

weight = tf.Variable(tf.truncated_normal([num_neurons, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.0, shape=[target.get_shape()[1]]))

In [10]:
y = tf.matmul(last, weight) + bias
prediction = tf.nn.softmax(y)

In [11]:
# Functions definition 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, target))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
minimize = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(target, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [12]:
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

In [13]:
batch_size = 400
no_of_batches = int(len(X_train) / batch_size)
epochs = 4000

loss_train_array = []
train_accuracy_array = []
test_accuracy_array = []

In [ ]:
with sess.as_default():
    
    for i in range(epochs):
        
        ptr = 0
        for j in range(no_of_batches):
            inp, out = X_train[ptr:ptr + batch_size], Y_train[ptr:ptr + batch_size]
            ptr += batch_size
            _, loss_train = sess.run([minimize, loss], {data: inp, target: out})
        

        if i % 100 == 0:
            train_accuracy = sess.run(
                accuracy, feed_dict={data: X_train, target: Y_train})
            train_accuracy_array.append(train_accuracy)

            test_accuracy = sess.run(
                accuracy, feed_dict={data: X_test, target: Y_test})
            test_accuracy_array.append(test_accuracy)

            print (
                'Epoch %04d, '
                'loss train %.8f, train accuracy %.8f, test accuracy %.8f'
                %
                (i,
                 loss_train,
                 train_accuracy,
                 test_accuracy))

Epoch 0000, loss train 3.96834111, train accuracy 0.08500000, test accuracy 0.09555347
Epoch 0100, loss train 0.01597336, train accuracy 0.99250001, test accuracy 0.11893025
Epoch 0200, loss train 0.01089315, train accuracy 0.99250001, test accuracy 0.11709560
Epoch 0300, loss train 0.00988228, train accuracy 0.99250001, test accuracy 0.11587585
Epoch 0400, loss train 0.00948339, train accuracy 0.99250001, test accuracy 0.11559359
Epoch 0500, loss train 0.00927098, train accuracy 0.99250001, test accuracy 0.11494844
Epoch 0600, loss train 0.00914481, train accuracy 0.99250001, test accuracy 0.11431336
Epoch 0700, loss train 0.00906032, train accuracy 0.99250001, test accuracy 0.11415207
Epoch 0800, loss train 0.00900305, train accuracy 0.99250001, test accuracy 0.11337587
Epoch 0900, loss train 0.00895426, train accuracy 0.99250001, test accuracy 0.11324482
Epoch 1000, loss train 0.00891881, train accuracy 0.99250001, test accuracy 0.11293233
Epoch 1100, loss train 0.00889281, train ac

In [19]:
# sample from the model now and then
preds = []
ix = X_train[:1]

for i in range(1000):
    pred = sess.run(prediction ,{data: ix.reshape(1, seq_length, 72)})
    index = np.argmax(pred, 1)
    
    preds.append(ix_to_char[index[0]])
    
    zero_array = np.zeros(72)
    zero_array[index] = 1
    
    tail = np.append(np.array(ix[0][1:]), np.array(zero_array.reshape(1, 72)))
    
    ix = tail.reshape(1, seq_length, 72)

In [16]:
print("".join(preds))

Veleta

Viento del Sur,
moreno, ardiente,
llegas sobre mi carne
tembloroso de auroras
boreales,
con tu capa de espectros
capitanes,
y riyéndote a gritos
del Dante.
¡Oh pulidor de estrellas!
Pero vienes
demasiado tarde.
Mi almario está musgoso
y he perdido la llave.

Sin ningún viento,
¡hazme caso!,
gira, corazón.

Brisas, gnomos y vientos
de ninguna parte.
Mosquitos de la rosa
de pétalos pirámides.
Alisios destetados
entre los rudos árboles,
flautass,
ydcae  cojatpa ddem   lasla
de bidalantdeshfelal
tasbsrmm  vanoo
tvoo n iree,
¡oéhV  rosdca rdstas

memapidonrád blósr
llalársblslml cientes
denntusgo as e rarlos,
deraétodos dne turos!
goora irne 
el leas,bnmrrodo e arluras
boemlrasoo  e iuit,
eo hzranas
e!b,imrast,,cerdn, ryyho posdendálaalslllal pirádees
.blriies.
tdlmesss brosoros á iumoscasarte,sdpadbula roa
e brelasos
detaide talmrdss
ddal eetat
lncone iejlét eusuayso,!gorana  aolla e.
Silnvntre MMerda de dulla ase.
Piresd, ppiaiios,b ipo ard
 te e!
llas nn gurossce de tacase
de ppa